## Imports, etc

In [ ]:
#@title Dependencies
!pip install tree-sitter
!git clone https://github.com/tree-sitter/tree-sitter-go
!git clone https://github.com/tree-sitter/tree-sitter-javascript
!git clone https://github.com/tree-sitter/tree-sitter-python
!git clone https://github.com/tree-sitter/tree-sitter-php
!git clone https://github.com/tree-sitter/tree-sitter-java
!git clone https://github.com/tree-sitter/tree-sitter-ruby
!git clone https://github.com/r-lib/tree-sitter-r.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tree-sitter: filename=tree_sitter-0.20.1-cp310-cp310-linux_x86_64.whl size=434921 sha256=11fee592ba42937d26c25dcb4b2ff7cdab90399c25ce754d8c77054b29d1644f
  Stored in directory: /root/.cache/pip/wheels/e6/d0/7a/a108b30f6615a71ca3a07ced1b149509d437a60c9d64820723
Successfully built tree-sitter
Cloning into 'tree-sitter-go'...
remote: Enumerating objects: 1805, done.
remote: Counting objects: 100% (1804/1804), done.
remote: Compressing objects: 100% (631/631), done.
remote: Total 1805 (delta 1164), reused 1741 (delta 1126), pack-reused 1
Receiving objects: 100% (1805/1805), 8.51 MiB | 17.81 MiB/s, done.
Resolving deltas: 100% (1164/1164), done.
Cloning into 'tr

## Scripts to extract identifiers

In [ ]:
from tree_sitter import Language, Parser

Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    '/content/tree-sitter-go',
    '/content/tree-sitter-javascript',
    '/content/tree-sitter-python',
    '/content/tree-sitter-r',
    '/content/tree-sitter-php',
    '/content/tree-sitter-java', 
    '/content/tree-sitter-ruby'
    
  ]
)

ModuleNotFoundError: ignored

Tree-sitter build failing for C, C++ in Github

Seems like it doesn't work for javascript

In [ ]:
go = Language('build/my-languages.so', 'go')
js = Language('build/my-languages.so', 'javascript')
py = Language('build/my-languages.so', 'python')
r = Language('build/my-languages.so', 'r')
php = Language('build/my-languages.so', 'php')
java = Language('build/my-languages.so', 'java')
ruby = Language('build/my-languages.so', 'ruby')

In [ ]:
parser = Parser()

In [ ]:
import re
from io import StringIO
import tokenize

def remove_comments_and_docstrings(source, lang):
    if lang in ['python']:
        """
        Returns 'source' minus comments and docstrings.
        """
        io_obj = StringIO(source)
        out = ""
        prev_toktype = tokenize.INDENT
        last_lineno = -1
        last_col = 0
        for tok in tokenize.generate_tokens(io_obj.readline):
            token_type = tok[0]
            token_string = tok[1]
            start_line, start_col = tok[2]
            end_line, end_col = tok[3]
            ltext = tok[4]
            if start_line > last_lineno:
                last_col = 0
            if start_col > last_col:
                out += (" " * (start_col - last_col))
            # Remove comments:
            if token_type == tokenize.COMMENT:
                pass
            # This series of conditionals removes docstrings:
            elif token_type == tokenize.STRING:
                if prev_toktype != tokenize.INDENT:
                    # This is likely a docstring; double-check we're not inside an operator:
                    if prev_toktype != tokenize.NEWLINE:
                        if start_col > 0:
                            out += token_string
            else:
                out += token_string
            prev_toktype = token_type
            last_col = end_col
            last_lineno = end_line
        temp = []
        for x in out.split('\n'):
            if x.strip() != "":
                temp.append(x)
        return '\n'.join(temp)
    elif lang in ['ruby']:
        return source
    else:
        def replacer(match):
            s = match.group(0)
            if s.startswith('/'):
                return " "  # note: a space and not an empty string
            else:
                return s

        pattern = re.compile(
            r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
            re.DOTALL | re.MULTILINE
        )
        temp = []
        for x in re.sub(pattern, replacer, source).split('\n'):
            if x.strip() != "":
                temp.append(x)
        return '\n'.join(temp)

In [ ]:
def tree_to_token_index(root_node):
    if (len(root_node.children) == 0 or root_node.type in ['string_literal', 'string',
                                                           'character_literal']) and root_node.type != 'comment':
        return [(root_node.start_point, root_node.end_point, root_node.type)]
    else:
        code_tokens = []
        for child in root_node.children:
            code_tokens += tree_to_token_index(child)
        return code_tokens

In [ ]:

def tree_to_variable_index(root_node, index_to_code):
    if (len(root_node.children) == 0 or root_node.type in ['string_literal', 'string',
                                                           'character_literal']) and root_node.type != 'comment':
        index = (root_node.start_point, root_node.end_point)
        _, code = index_to_code[index]
        if root_node.type != code:
            return [(root_node.start_point, root_node.end_point)]
        else:
            return []
    else:
        code_tokens = []
        for child in root_node.children:
            code_tokens += tree_to_variable_index(child, index_to_code)
        return code_tokens

In [ ]:
def index_to_code_token(index, code):
    start_point = index[0]
    end_point = index[1]
    if start_point[0] == end_point[0]:
        s = code[start_point[0]][start_point[1]:end_point[1]]
    else:
        s = ""
        s += code[start_point[0]][start_point[1]:]
        for i in range(start_point[0] + 1, end_point[0]):
            s += code[i]
        s += code[end_point[0]][:end_point[1]]
    return s

In [ ]:
def get_type(parser, code, lang):
    parser.set_language(lang)
    code = remove_comments_and_docstrings(code,lang)
    tree = parser.parse(bytes(code, 'utf8'))
    root_node = tree.root_node
    tokens_index = tree_to_token_index(root_node)
    code = code.split('\n')
    code_tokens = [index_to_code_token(x, code) for x in tokens_index]
    index_to_code = {}
    for (index, code) in zip(tokens_index, code_tokens):
        index_to_code[index] = code
    return index_to_code

In [ ]:
def get_identifiers(parser, code, lang):
    parser.set_language(lang)
    code = remove_comments_and_docstrings(code,lang)
    tree = parser.parse(bytes(code, 'utf8'))
    root_node = tree.root_node
    tokens_index = tree_to_token_index(root_node)
    code = code.split('\n')
    code_tokens = [index_to_code_token(x, code) for x in tokens_index]
    index_to_code = {}
    for (index, code) in zip(tokens_index, code_tokens):
        if(index[2]=='identifier'):
            index_to_code[index] = code
    return index_to_code

## code samples from python, javascript & R

In [ ]:
code_set = [
#Python
("""
# Python program to swap two variables

x = 5
y = 10

# To take inputs from the user
#x = input('Enter value of x: ')
#y = input('Enter value of y: ')

# create a temporary variable and swap the values
temp = x
x = y
y = temp

print('The value of x after swapping: {}'.format(x))
print('The value of y after swapping: {}'.format(y))
def abd(a):
  return a
""", py),

#javascript
("""
<!DOCTYPE html>
<html>
<body>

<h2>Assigning JavaScript Values</h2>

<p>In JavaScript the = operator is used to assign values to variables.</p>

<p id="demo"></p>

<script>
let x, y;
x = 5;
y = 6;
document.getElementById("demo").innerHTML = x + y;
</script>

</body>
</html>
""", js),

("""
//JavaScript program to swap two variables

//take input from the users
let a = prompt('Enter the first variable: ');
let b = prompt('Enter the second variable: ');

//create a temporary variable
let temp;

//swap variables
temp = a;
a = b;
b = temp;

console.log(`The value of a after swapping: ${a}`);
console.log(`The value of b after swapping: ${b}`);
""", js),

#R
("""
#utility functions

readinteger <- function()
{ 
  n <- readline(prompt="Enter an integer: ")
  if(!grepl("^[0-9]+$",n))
  {
    return(readinteger())
  }
  return(as.integer(n))
}

# real program start here
  
num <- round(runif(1) * 100, digits = 0)
guess <- -1

cat("Guess a number between 0 and 100.\n")

while(guess != num)
{ 
  guess <- readinteger()
  if (guess == num)
  {
    cat("Congratulations,", num, "is right.\n")
  }
  else if (guess < num)
  {
    cat("It's bigger!\n")
  }
  else if(guess > num)
  {
    cat("It's smaller!\n")
  }
}
""", r),

("""
html_extras_for_document <- function(knit_meta, runtime, dependency_resolver,
                                     format_deps = NULL) {

  extras <- list()

  # merge the dependencies discovered with the dependencies of this format and
  # dependencies discovered in knit_meta
  all_dependencies <- if (is.null(format_deps)) list() else format_deps
  all_dependencies <- append(all_dependencies, flatten_html_dependencies(knit_meta))
  extras$dependencies <- dependency_resolver(all_dependencies)

  # return extras
  extras
}

""", r),

#Go
("""
package main

import "fmt"

func main() {

    var a [5]int
    fmt.Println("emp:", a)

    a[4] = 100
    fmt.Println("set:", a)
    fmt.Println("get:", a[4])

    fmt.Println("len:", len(a))

    b := [5]int{1, 2, 3, 4, 5}
    fmt.Println("dcl:", b)

    var twoD [2][3]int
    for i := 0; i < 2; i++ {
        for j := 0; j < 3; j++ {
            twoD[i][j] = i + j
        }
    }
    fmt.Println("2d: ", twoD)
}
""",go),

#PHP 
(
"""
<?php
$name = "Quincy";
echo "Hi! My name is " . $name . "<br>";
echo "Hi! My name is " . $NAME . "<br>";
echo "Hi! My name is " . $NaMe . "<br>";
?>
""", php),

#Ruby
(
"""
a = [ 45, 3, 19, 8 ]
b = [ 'sam', 'max', 56, 98.9, 3, 10, 'jill' ]
print (a + b).join(' '), "\n"
print a[2], " ", b[4], " ", b[-2], "\n"
print a.sort.join(' '), "\n"
a << 57 << 9 << 'phil'
print "A: ", a.join(' '), "\n"

b << 'alex' << 48 << 220
print "B: ", b.join(' '), "\n"
print "pop: ", b.pop, "\n"
print "shift: ", b.shift, "\n"
print "C: ", b.join(' '), "\n"

b.delete_at(2)
b.delete('alex')
print "D: ", b.join(' '), "\n"
""", ruby),

#java
("""
/* CallingMethodsInSameClass.java
 *
 * illustrates how to call static methods a class
 * from a method in the same class
 */

public class CallingMethodsInSameClass
{
	public static void main(String[] args) {
		printOne();
		printOne();
		printTwo();
	}

	public static void printOne() {
		System.out.println("Hello World");
	}

	public static void printTwo() {
		printOne();
		printOne();
	}
}
""", java),

]


In [ ]:

import pprint
for code, lang in code_set:
    print('*'*100)
    print("Original Code")
    print(code)
    print("_"*100)
    print("Parsed Identifiers")
    pprint.pprint(get_type(parser, code, lang))
    print('*'*100)

****************************************************************************************************
Original Code

# Python program to swap two variables

x = 5
y = 10

# To take inputs from the user
#x = input('Enter value of x: ')
#y = input('Enter value of y: ')

# create a temporary variable and swap the values
temp = x
x = y
y = temp

print('The value of x after swapping: {}'.format(x))
print('The value of y after swapping: {}'.format(y))
def abd(a):
  return a

____________________________________________________________________________________________________
Parsed Identifiers
{((1, 0), (1, 1), 'identifier'): 'x',
 ((1, 2), (1, 3), '='): '=',
 ((1, 4), (1, 5), 'integer'): '5',
 ((2, 0), (2, 1), 'identifier'): 'y',
 ((2, 2), (2, 3), '='): '=',
 ((2, 4), (2, 6), 'integer'): '10',
 ((7, 0), (7, 4), 'identifier'): 'temp',
 ((7, 5), (7, 6), '='): '=',
 ((7, 7), (7, 8), 'identifier'): 'x',
 ((8, 0), (8, 1), 'identifier'): 'x',
 ((8, 2), (8, 3), '='): '=',
 ((8, 4), (8, 5), 'identifi

## Extracted Identifiers

**Parsed identifier contains the following**

((line #, start index), (line #, end index), identifier): name of identifier



In [ ]:
import pprint
for code, lang in code_set:
    print('*'*100)
    print("Original Code")
    print(code)
    print("_"*100)
    print("Parsed Identifiers")
    pprint.pprint(get_identifiers(parser, code, lang))
    print('*'*100)

****************************************************************************************************
Original Code

# Python program to swap two variables

x = 5
y = 10

# To take inputs from the user
#x = input('Enter value of x: ')
#y = input('Enter value of y: ')

# create a temporary variable and swap the values
temp = x
x = y
y = temp

print('The value of x after swapping: {}'.format(x))
print('The value of y after swapping: {}'.format(y))
def abd(a):
  return a

____________________________________________________________________________________________________
Parsed Identifiers
{((1, 0), (1, 1), 'identifier'): 'x',
 ((2, 0), (2, 1), 'identifier'): 'y',
 ((7, 0), (7, 4), 'identifier'): 'temp',
 ((7, 7), (7, 8), 'identifier'): 'x',
 ((8, 0), (8, 1), 'identifier'): 'x',
 ((8, 4), (8, 5), 'identifier'): 'y',
 ((9, 0), (9, 1), 'identifier'): 'y',
 ((9, 4), (9, 8), 'identifier'): 'temp',
 ((10, 0), (10, 5), 'identifier'): 'print',
 ((10, 42), (10, 48), 'identifier'): 'format',
 ((10,

## Types identified by tree-sitter for Python

extras, conflicts, supertypes, externals, inline, word, module, _statement, _simple_statements, _simple_statement, import_statement, import_prefix, relative_import, future_import_statement, import_from_statement, _import_list, aliased_import, wildcard_import, print_statement, chevron, assert_statement, expression_statement, named_expression, _named_expression_lhs, return_statement, delete_statement, _expressions, raise_statement, pass_statement, break_statement, continue_statement, _compound_statement, if_statement, elif_clause, else_clause, match_statement, case_clause, for_statement, while_statement, try_statement, except_clause, except_group_clause, finally_clause, with_statement, with_clause, with_item, function_definition, parameters, lambda_parameters, list_splat, dictionary_splat, global_statement, nonlocal_statement, exec_statement, class_definition, parenthesized_list_splat, argument_list, decorated_definition, decorator, _suite, block, expression_list, dotted_name, _parameters, _patterns, parameter, pattern, tuple_pattern, list_pattern, default_parameter, typed_default_parameter, list_splat_pattern, dictionary_splat_pattern, as_pattern, _expression_within_for_in_clause, expression, primary_expression, not_operator, boolean_operator, binary_operator, unary_operator, comparison_operator, lambda, lambda_within_for_in_clause, assignment, augmented_assignment, _left_hand_side, pattern_list, _right_hand_side, yield, attribute, subscript, slice, ellipsis, call, typed_parameter, type, keyword_argument, list, set, tuple, dictionary, pair, list_comprehension, dictionary_comprehension, set_comprehension, generator_expression, _comprehension_clauses, parenthesized_expression, _collection_elements, for_in_clause, if_clause, conditional_expression, concatenated_string, string, string_content, interpolation, _f_expression, _escape_interpolation, escape_sequence, _not_escape_sequence, format_specifier, type_conversion, integer, float, identifier, keyword_identifier, true, false, none, await, comment, positional_separator, keyword_separator, commaSep1, sep1

## Types identified by tree-sitter for Ruby

inline, externals, extras, word, supertypes, program, uninterpreted, block_body, _statements, begin_block, end_block, _statement, method, singleton_method, _method_rest, rescue_modifier_arg, rescue_modifier_expression, _body_expr, parameters, bare_parameters, block_parameters, _formal_parameter, _simple_formal_parameter, forward_parameter, splat_parameter, hash_splat_parameter, hash_splat_nil, block_parameter, keyword_parameter, optional_parameter, class, superclass, singleton_class, module, return_command, yield_command, break_command, next_command, return, yield, break, next, redo, retry, if_modifier, unless_modifier, while_modifier, until_modifier, rescue_modifier, while, until, for, in, do, case, case_match, when, in_clause, pattern, _guard, if_guard, unless_guard, _pattern_top_expr_body, _array_pattern_n, _pattern_expr, as_pattern, _pattern_expr_alt, alternative_pattern, _array_pattern_body, array_pattern, _find_pattern_body, find_pattern, _hash_pattern_body, keyword_pattern, _hash_pattern_any_rest, hash_pattern, _pattern_expr_basic, parenthesized_pattern, _pattern_value, _pattern_range, _pattern_primitive, _pattern_lambda, _pattern_literal, _keyword_variable, line, file, encoding, variable_reference_pattern, expression_reference_pattern, _pattern_constant, _pattern_constant_resolution, if, unless, elsif, else, then, begin, ensure, rescue, exceptions, exception_variable, body_statement, _body_statement, _expression, match_pattern, test_pattern, _arg, _unary_minus_pow, _pow, _primary, parenthesized_statements, element_reference, scope_resolution, _call_operator, _call, command_call, command_call_with_block, _chained_command_call, call, command_argument_list, argument_list, _argument_list_with_trailing_comma, _argument, forward_argument, splat_argument, hash_splat_argument, block_argument, do_block, block, _arg_rhs, assignment, command_assignment, operator_assignment, command_operator_assignment, conditional, range, binary, command_binary, unary, command_unary, parenthesized_unary, unary_literal, _literal, _numeric, _simple_numeric, right_assignment_list, left_assignment_list, _mlhs, destructured_left_assignment, rest_assignment, _function_identifier, _function_identifier_call, _lhs, _variable, operator, _method_name, _nonlocal_variable, setter, undef, alias, comment, integer, _int_or_float, float, complex, rational, super, self, true, false, nil, constant, constant_suffix, identifier, identifier_suffix, instance_variable, class_variable, global_variable, chained_string, character, interpolation, string, subshell, string_array, symbol_array, delimited_symbol, regex, heredoc_body, _literal_contents, escape_sequence, array, hash, pair, lambda, empty_statement, _terminator, sep, sep1, commaSep1, commaSep

## Types identified by tree-sitter for java

, extras, supertypes, inline, conflicts, word, program, _literal, decimal_integer_literal, hex_integer_literal, octal_integer_literal, binary_integer_literal, decimal_floating_point_literal, hex_floating_point_literal, true, false, character_literal, string_literal, _string_literal, _multiline_string_literal, string_fragment, _multiline_string_fragment, _escape_sequence, escape_sequence, null_literal, expression, cast_expression, assignment_expression, binary_expression, instanceof_expression, lambda_expression, inferred_parameters, ternary_expression, unary_expression, update_expression, primary_expression, array_creation_expression, dimensions_expr, parenthesized_expression, condition, class_literal, object_creation_expression, _unqualified_object_creation_expression, field_access, array_access, method_invocation, argument_list, method_reference, type_arguments, wildcard, _wildcard_bounds, dimensions, switch_expression, switch_block, switch_block_statement_group, switch_rule, switch_label, statement, block, expression_statement, labeled_statement, assert_statement, do_statement, break_statement, continue_statement, return_statement, yield_statement, synchronized_statement, throw_statement, try_statement, catch_clause, catch_formal_parameter, catch_type, finally_clause, try_with_resources_statement, resource_specification, resource, if_statement, while_statement, for_statement, enhanced_for_statement, _annotation, marker_annotation, annotation, annotation_argument_list, element_value_pair, _element_value, element_value_array_initializer, declaration, module_declaration, module_body, module_directive, requires_module_directive, requires_modifier, exports_module_directive, opens_module_directive, uses_module_directive, provides_module_directive, package_declaration, import_declaration, asterisk, enum_declaration, enum_body, enum_body_declarations, enum_constant, class_declaration, modifiers, type_parameters, type_parameter, type_bound, superclass, super_interfaces, type_list, permits, class_body, _class_body_declaration, static_initializer, constructor_declaration, _constructor_declarator, constructor_body, explicit_constructor_invocation, _name, scoped_identifier, field_declaration, record_declaration, annotation_type_declaration, annotation_type_body, annotation_type_element_declaration, _default_value, interface_declaration, extends_interfaces, interface_body, constant_declaration, _variable_declarator_list, variable_declarator, _variable_declarator_id, _variable_initializer, array_initializer, _type, _unannotated_type, _simple_type, annotated_type, scoped_type_identifier, generic_type, array_type, integral_type, floating_point_type, boolean_type, void_type, _method_header, _method_declarator, formal_parameters, formal_parameter, receiver_parameter, spread_parameter, throws, local_variable_declaration, method_declaration, compact_constructor_declaration, _reserved_identifier, this, super, identifier, comment, line_comment, block_comment, sep1, commaSep1, commaSep

## Types identified by tree-sitter for javascript

, externals, extras, supertypes, inline, precedences, conflicts, word, program, hash_bang_line, export_statement, namespace_export, export_clause, export_specifier, _module_export_name, declaration, import, import_statement, import_clause, _from_clause, namespace_import, named_imports, import_specifier, statement, expression_statement, variable_declaration, lexical_declaration, variable_declarator, statement_block, else_clause, if_statement, switch_statement, for_statement, for_in_statement, _for_header, while_statement, do_statement, try_statement, with_statement, break_statement, continue_statement, debugger_statement, return_statement, throw_statement, empty_statement, labeled_statement, switch_body, switch_case, switch_default, catch_clause, finally_clause, parenthesized_expression, _expressions, expression, primary_expression, yield_expression, object, object_pattern, assignment_pattern, object_assignment_pattern, array, array_pattern, glimmer_template, _glimmer_template_content, glimmer_opening_tag, glimmer_closing_tag, _jsx_element, jsx_element, jsx_fragment, jsx_text, jsx_expression, _jsx_child, jsx_opening_element, jsx_identifier, _jsx_identifier, nested_identifier, jsx_namespace_name, _jsx_element_name, jsx_closing_element, jsx_self_closing_element, _jsx_attribute, _jsx_attribute_name, jsx_attribute, _jsx_attribute_value, class, class_declaration, class_heritage, function, function_declaration, generator_function, generator_function_declaration, arrow_function, _call_signature, _formal_parameter, optional_chain, call_expression, new_expression, await_expression, member_expression, subscript_expression, _lhs_expression, assignment_expression, _augmented_assignment_lhs, augmented_assignment_expression, _initializer, _destructuring_pattern, spread_element, ternary_expression, binary_expression, unary_expression, update_expression, sequence_expression, string, unescaped_double_string_fragment, unescaped_single_string_fragment, escape_sequence, comment, template_string, template_substitution, regex, regex_pattern, regex_flags, number, _identifier, identifier, private_property_identifier, meta_property, this, super, true, false, null, undefined, arguments, decorator, decorator_member_expression, decorator_call_expression, class_body, field_definition, formal_parameters, class_static_block, pattern, rest_pattern, method_definition, pair, pair_pattern, _property_name, computed_property_name, _reserved_identifier, _semicolon, commaSep1, commaSep

## Types identified by tree-sitter for PHP

, externals, supertypes, word, conflicts, inline, extras, program, php_tag, text_interpolation, text, _statement, empty_statement, reference_modifier, function_static_declaration, static_variable_declaration, global_declaration, namespace_definition, namespace_use_declaration, namespace_use_clause, qualified_name, namespace_name_as_prefix, namespace_name, namespace_aliasing_clause, namespace_use_group, namespace_use_group_clause, trait_declaration, interface_declaration, base_clause, enum_declaration, enum_declaration_list, _enum_member_declaration, enum_case, class_declaration, declaration_list, final_modifier, abstract_modifier, readonly_modifier, class_interface_clause, _member_declaration, const_declaration, _class_const_declaration, _const_declaration, property_declaration, _modifier, property_element, property_initializer, method_declaration, var_modifier, static_modifier, use_declaration, use_list, use_instead_of_clause, use_as_clause, visibility_modifier, function_definition, _function_definition_header, _arrow_function_header, arrow_function, formal_parameters, property_promotion_parameter, simple_parameter, variadic_parameter, _type, _types, named_type, optional_type, bottom_type, union_type, intersection_type, primitive_type, cast_type, _return_type, const_element, echo_statement, unset_statement, declare_statement, declare_directive, _literal, float, try_statement, catch_clause, type_list, finally_clause, goto_statement, continue_statement, break_statement, integer, return_statement, throw_expression, while_statement, do_statement, for_statement, _expressions, sequence_expression, foreach_statement, foreach_pair, _foreach_value, if_statement, colon_block, else_if_clause, else_clause, else_if_clause_2, else_clause_2, match_expression, match_block, match_condition_list, match_conditional_expression, match_default_expression, switch_statement, switch_block, case_statement, default_statement, compound_statement, named_label_statement, expression_statement, _expression, _unary_expression, unary_op_expression, exponentiation_expression, clone_expression, _primary_expression, parenthesized_expression, class_constant_access_expression, print_intrinsic, anonymous_function_creation_expression, anonymous_function_use_clause, object_creation_expression, _class_type_designator, update_expression, cast_expression, cast_variable, assignment_expression, reference_assignment_expression, conditional_expression, augmented_assignment_expression, _variable, member_access_expression, nullsafe_member_access_expression, scoped_property_access_expression, list_literal, _list_destructing, _array_destructing, _array_destructing_element, _callable_variable, function_call_expression, _callable_expression, parenthesized_expression, scoped_call_expression, _scope_resolution_qualifier, relative_scope, variadic_placeholder, arguments, argument, member_call_expression, nullsafe_member_call_expression, variadic_unpacking, _member_name, subscript_expression, _dereferencable_expression, array_creation_expression, attribute_group, attribute_list, attribute, _complex_string_part, _simple_string_member_access_expression, _simple_string_subscript_unary_expression, _simple_string_array_access_argument, _simple_string_subscript_expression, _simple_string_part, escape_sequence, _interpolated_string_body, _interpolated_string_body_heredoc, encapsed_string, string, string_value, heredoc_body, heredoc, _new_line, nowdoc_body, nowdoc, _interpolated_execution_operator_body, shell_command_expression, boolean, null, _string, dynamic_variable_name, _variable_name, variable_name, variable_reference, by_ref, yield_expression, array_element_initializer, binary_expression, include_expression, include_once_expression, require_expression, require_once_expression, name, _reserved_identifier, comment, _semicolon, keyword, commaSep1, commaSep, pipeSep1, ampSep1


## Types identified by tree-sitter for go

, extras, inline, word, conflicts, supertypes, source_file, _top_level_declaration, package_clause, import_declaration, import_spec, dot, blank_identifier, import_spec_list, _declaration, const_declaration, const_spec, var_declaration, var_spec, function_declaration, method_declaration, type_parameter_list, parameter_list, parameter_declaration, variadic_parameter_declaration, type_alias, type_declaration, type_spec, field_name_list, expression_list, _type, parenthesized_type, _simple_type, generic_type, type_arguments, pointer_type, array_type, implicit_length_array_type, slice_type, struct_type, field_declaration_list, field_declaration, interface_type, _interface_body, interface_type_name, constraint_elem, constraint_term, struct_elem, struct_term, method_spec, map_type, channel_type, function_type, block, _statement_list, _statement, empty_statement, _simple_statement, send_statement, receive_statement, inc_statement, dec_statement, assignment_statement, short_var_declaration, labeled_statement, empty_labeled_statement, fallthrough_statement, break_statement, continue_statement, goto_statement, return_statement, go_statement, defer_statement, if_statement, for_statement, for_clause, range_clause, expression_switch_statement, expression_case, default_case, type_switch_statement, _type_switch_header, type_case, select_statement, communication_case, _expression, parenthesized_expression, call_expression, variadic_argument, special_argument_list, argument_list, selector_expression, index_expression, slice_expression, type_assertion_expression, type_conversion_expression, composite_literal, literal_value, literal_element, keyed_element, func_literal, unary_expression, binary_expression, qualified_type, identifier, _type_identifier, _field_identifier, _package_identifier, _string_literal, raw_string_literal, interpreted_string_literal, _interpreted_string_literal_basic_content, escape_sequence, int_literal, float_literal, imaginary_literal, rune_literal, nil, true, false, iota, comment, commaSep1, commaSep


## ignore

In [ ]:
id = """
func
extras
func
inline
func
word
func
conflicts
func
supertypes
func
source_file
func
_top_level_declaration
func
package_clause
func
import_declaration
func
import_spec
func
dot
func
blank_identifier
func
import_spec_list
func
_declaration
func
const_declaration
func
const_spec
func
var_declaration
func
var_spec
func
function_declaration
func
method_declaration
func
type_parameter_list
func
parameter_list
func
parameter_declaration
func
variadic_parameter_declaration
func
type_alias
func
type_declaration
func
type_spec
func
field_name_list
func
expression_list
func
_type
func
parenthesized_type
func
_simple_type
func
generic_type
func
type_arguments
func
pointer_type
func
array_type
func
implicit_length_array_type
func
slice_type
func
struct_type
func
field_declaration_list
func
field_declaration
func
interface_type
func
_interface_body
func
interface_type_name
func
constraint_elem
func
constraint_term
func
struct_elem
func
struct_term
func
method_spec
func
map_type
func
channel_type
func
function_type
func
block
func
_statement_list
func
_statement
func
empty_statement
func
_simple_statement
func
send_statement
func
receive_statement
func
inc_statement
func
dec_statement
func
assignment_statement
func
short_var_declaration
func
labeled_statement
func
empty_labeled_statement
func
fallthrough_statement
func
break_statement
func
continue_statement
func
goto_statement
func
return_statement
func
go_statement
func
defer_statement
func
if_statement
func
for_statement
func
for_clause
func
range_clause
func
expression_switch_statement
func
expression_case
func
default_case
func
type_switch_statement
func
_type_switch_header
func
type_case
func
select_statement
func
communication_case
func
_expression
func
parenthesized_expression
func
call_expression
func
variadic_argument
func
special_argument_list
func
argument_list
func
selector_expression
func
index_expression
func
slice_expression
func
type_assertion_expression
func
type_conversion_expression
func
composite_literal
func
literal_value
func
literal_element
func
keyed_element
func
func_literal
func
unary_expression
func
binary_expression
func
qualified_type
func
identifier
func
_type_identifier
func
_field_identifier
func
_package_identifier
func
_string_literal
func
raw_string_literal
func
interpreted_string_literal
func
_interpreted_string_literal_basic_content
func
escape_sequence
func
int_literal
func
float_literal
func
imaginary_literal
func
rune_literal
func
nil
func
true
func
false
func
iota
func
comment
func
commaSep1
func
commaSep"""

id = id.split("\n")
id


['',
 'func',
 'extras',
 'func',
 'inline',
 'func',
 'word',
 'func',
 'conflicts',
 'func',
 'supertypes',
 'func',
 'source_file',
 'func',
 '_top_level_declaration',
 'func',
 'package_clause',
 'func',
 'import_declaration',
 'func',
 'import_spec',
 'func',
 'dot',
 'func',
 'blank_identifier',
 'func',
 'import_spec_list',
 'func',
 '_declaration',
 'func',
 'const_declaration',
 'func',
 'const_spec',
 'func',
 'var_declaration',
 'func',
 'var_spec',
 'func',
 'function_declaration',
 'func',
 'method_declaration',
 'func',
 'type_parameter_list',
 'func',
 'parameter_list',
 'func',
 'parameter_declaration',
 'func',
 'variadic_parameter_declaration',
 'func',
 'type_alias',
 'func',
 'type_declaration',
 'func',
 'type_spec',
 'func',
 'field_name_list',
 'func',
 'expression_list',
 'func',
 '_type',
 'func',
 'parenthesized_type',
 'func',
 '_simple_type',
 'func',
 'generic_type',
 'func',
 'type_arguments',
 'func',
 'pointer_type',
 'func',
 'array_type',
 'func',
 'im

In [ ]:
id1 = [id[x] for x in range(0, len(id)+1, 2) ]
id1

['',
 'extras',
 'inline',
 'word',
 'conflicts',
 'supertypes',
 'source_file',
 '_top_level_declaration',
 'package_clause',
 'import_declaration',
 'import_spec',
 'dot',
 'blank_identifier',
 'import_spec_list',
 '_declaration',
 'const_declaration',
 'const_spec',
 'var_declaration',
 'var_spec',
 'function_declaration',
 'method_declaration',
 'type_parameter_list',
 'parameter_list',
 'parameter_declaration',
 'variadic_parameter_declaration',
 'type_alias',
 'type_declaration',
 'type_spec',
 'field_name_list',
 'expression_list',
 '_type',
 'parenthesized_type',
 '_simple_type',
 'generic_type',
 'type_arguments',
 'pointer_type',
 'array_type',
 'implicit_length_array_type',
 'slice_type',
 'struct_type',
 'field_declaration_list',
 'field_declaration',
 'interface_type',
 '_interface_body',
 'interface_type_name',
 'constraint_elem',
 'constraint_term',
 'struct_elem',
 'struct_term',
 'method_spec',
 'map_type',
 'channel_type',
 'function_type',
 'block',
 '_statement_list

In [ ]:
(', ').join(id1)

', extras, inline, word, conflicts, supertypes, source_file, _top_level_declaration, package_clause, import_declaration, import_spec, dot, blank_identifier, import_spec_list, _declaration, const_declaration, const_spec, var_declaration, var_spec, function_declaration, method_declaration, type_parameter_list, parameter_list, parameter_declaration, variadic_parameter_declaration, type_alias, type_declaration, type_spec, field_name_list, expression_list, _type, parenthesized_type, _simple_type, generic_type, type_arguments, pointer_type, array_type, implicit_length_array_type, slice_type, struct_type, field_declaration_list, field_declaration, interface_type, _interface_body, interface_type_name, constraint_elem, constraint_term, struct_elem, struct_term, method_spec, map_type, channel_type, function_type, block, _statement_list, _statement, empty_statement, _simple_statement, send_statement, receive_statement, inc_statement, dec_statement, assignment_statement, short_var_declaration, lab